In [55]:
import requests
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import json

client = OpenAI()
s = requests.Session()

GROUP_THEM_URL = "http://127.0.0.1:8080/groupthem"

### WordGuess

In [9]:
todo

{'guesses': [], 'maximum_guesses': 6}

### GroupThem

In [2]:
state = s.get(GROUP_THEM_URL).json()
print(state)
schema = s.get(GROUP_THEM_URL + "/schema").json()
print(schema)

{'guesses': [], 'available_words': ['echo', 'backbone', 'parrot', 'ham', 'cannonball', 'quote', 'pillar', 'bellyflop', 'talk', 'cornerstone', 'backflip', 'pirate', 'satellite', 'repeat', 'anchor', 'jackknife']}
{'$schema': 'http://json-schema.org/draft-07/schema#', 'title': 'GroupThemRequest', 'type': 'object', 'required': ['guess'], 'properties': {'guess': {'type': 'array', 'items': {'type': 'string'}}}}


In [ ]:
models = ["gpt-4o", "gpt-4o-2024-08-06", "gpt-4o-mini", "gpt-3.5-turbo"]

In [54]:
schema = '{"reasoning": String, "connections": [String; 4]}'

for model in models:

    conversation = [
        {"role": "system", "content": "You are a helpful assistant. Create a json object with the connection groups & your reasoning. ### Schema\n" + schema },
        {"role": "user", "content": "Create the mentioned JSON object through trying to find a connection between exactly 4 of these items: " + ", ".join(state["available_words"]) + "."},
    ]

    print(f"# Model: {model}")
    
    for i in range(10):
        # Ask model to attempt connection
        response = client.chat.completions.create(
            model=model,
            messages=conversation
        ).choices[0].message.content
        
        start = response.find("{")
        end = response.find("}", start)
        
        try:
            json_response = json.loads(response[start:end+1])
        except Exception as e:
            print(response)
            print(e)
            conversation.append({"role": "user", "content": "Invalid JSON object. Please try again following the schema: " + schema})
            
        print(f"\n>Guess: {json_response['connections']}\nReasoning: {json_response['reasoning']}")
        # Send connections
        payload = {"guess": json_response["connections"]}
        
        try:
            response = s.post(GROUP_THEM_URL, json=payload)
            if response.status_code != 200:
                
                if "guess already made" in response.text.lower():
                    conversation.append({"role": "user", "content": "WARNING: This guess was already made"})
                    print("Guess already made")
                    continue
                
                if "not a valid word" in response.text.lower():
                    conversation.append({"role": "user", "content": f"WARNING: {response.text}"})
                    print("Invalid word")
                    continue
                
                print(f"\n### response.text")
                break
            available_words = response.json()["available_words"]
        except Exception as e:
            print(e)
            break
        
        win = json_response["connections"][0] not in available_words
        print(f"Win: {win}\n")
        if len(available_words) == 0:
            print(f"### You won in {i+1} guesses!")
            break
        else:
            
            if win:
                conversation = [
                    {"role": "system", "content": "You are a helpful assistant. Create a json object with the connection groups & your reasoning. ### Schema\n" + schema },
                    {"role": "user", "content": "Create the mentioned JSON object through trying to find a connection between exactly 4 of these items: " + ", ".join(available_words) + "."},
                ]
            else:
                conversation.append({"role": "user", "content": f"You were INCORRECT. Try again with different options."})
    print("\n---\n")
            


>Guess: ['pirate', 'cannonball', 'anchor', 'parrot']
Reasoning: All the items can be related to actions or objects that are commonly associated with a pirate theme.
Win: False


>Guess: ['pirate', 'cannonball', 'anchor', 'backbone']
Reasoning: All these items are related to different physical actions or objects that could be found on a pirate ship.
Win: False

{
	"reasoning": "All items represent actions or objects related to a specific theme: "water activities".",
	"connections": ["bellyflop", "cannonball", "backflip", "jackknife"]
}
Expecting ',' delimiter: line 2 column 85 (char 86)

>Guess: ['pirate', 'cannonball', 'anchor', 'backbone']
Reasoning: All these items are related to different physical actions or objects that could be found on a pirate ship.
Guess already made

>Guess: ['cannonball', 'anchor', 'pirate', 'anchor']
Reasoning: All these items are related to actions or objects found in water or naval settings.
You have to guess 4 words
